In [3]:

import cftime
import xarray as xr
import datetime
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
import numpy as np
fn = '/home/graham/Downloads/treeFrac_Lmon_CESM2_land-hist_r1i1p1f1_gn_194901-201512.nc'

dset = xr.open_dataset(fn)
t = cftime.DatetimeNoLeap(2001, 1, 1)
#need to use longitude 0-360, convert from -180 to 180
ffrac = dset['treeFrac'].sel(lat=55,lon=235,time=t, method='nearest')
fp = r'Hansen_GFC-2020-v1.8_lossyear_60N_130W.tif'
tree_cover = r'Hansen_GFC-2020-v1.8_treecover2000_60N_130W.tif'
img = rasterio.open(fp)
tree_cover_img = rasterio.open(tree_cover)


In [67]:
band1 = img.read(1)
band2 = tree_cover_img.read(1)
#print(img.bounds.left, img.bounds.bottom, img.bounds.top,img.bounds.left)
x,y = img.index(-125,55.13089)
x1,y1 = img.index(-126.25,56.073299)


In [68]:
cropped = band1[x1:x,y1:y]
tree_cover_cropped = band2[x1:x, y1:y]
cropped[cropped != 1] = 0
tree_loss_2001 = tree_cover_cropped[cropped>0]
percentage_2001 = tree_loss_2001.sum() / cropped.size
percentage_2000 = tree_cover_cropped.sum() / cropped.size
percentage_2000

68.961984933687

In [81]:
t_2000 = cftime.DatetimeNoLeap(2000,1,1)
t_2001 = cftime.DatetimeNoLeap(2001, 1, 1)

cesm_output_2001 = dset['treeFrac'].sel(time=t_2001, method='nearest')
x = cesm_output_2001.where(cesm_output_2001 > 68.0, drop=True)
x = x.where(x < 70.0, drop=True)

x = x.where(np.logical_and(x.lat < 59,x.lat > 53), drop=True)
x = x.where(np.logical_and(x.lon < 240,x.lon > 230), drop=True)
x[0][0]

<xarray.DataArray 'treeFrac' ()>
array(nan, dtype=float32)
Coordinates:
    lat      float64 53.25
    lon      float64 231.2
    time     object 2001-01-16 03:00:00
Attributes:
    cell_measures:  area: areacella
    cell_methods:   area: mean where land over all_area_types time: mean
    comment:        fraction of entire grid cell  that is covered by trees.
    description:    fraction of entire grid cell  that is covered by trees.
    frequency:      mon
    id:             treeFrac
    long_name:      Tree Cover Percentage
    mipTable:       Lmon
    out_name:       treeFrac
    prov:           Lmon ((isd.003))
    realm:          land
    standard_name:  area_fraction
    time:           time
    time_label:     time-mean
    time_title:     Temporal mean
    title:          Tree Cover Percentage
    type:           real
    units:          %
    variable_id:    treeFrac

In [89]:
#I want to compute the difference between 2001 and 2000 to see if it lines up with GFC data. 

cesm_output_2000 = dset['treeFrac'].sel(time=t_2000, method='nearest')

difference = cesm_output_2001 - cesm_output_2000

difference.where(np.logical_and(difference < 0.3, difference > .1), drop=True)
###that there is data is very good!!! that means there is a quadrant that has the same change as what is recorded in 

<xarray.DataArray 'treeFrac' (lat: 72, lon: 124)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)
Coordinates:
  * lat      (lat) float64 -44.76 -42.88 -41.94 -38.17 ... 57.96 58.9 59.84
  * lon      (lon) float64 0.0 1.25 2.5 5.0 6.25 ... 355.0 356.2 357.5 358.8